# <center>Big Data &ndash; Exercise 1</center>
## <center>Fall 2023 &ndash; Week 1 &ndash; ETH Zurich</center>

Hongyu Hè honghe@inf.ethz.ch

### Aims
- **After this exercise:** 
    - Understand the SQL language and its common query patterns.
    - Understand the 'table' data shape, normalization, and when they can (and should) be used.
    - be able to query data in tables with the SQL language.
- **Later in the semester:** 
    - Relate these language features and query patterns relative to other data shapes, technologies, and the languages designed to query them.
    - Understand when tables are not the appropriate shape for your data and when you can (and should) throw normalization away!

### Prerequisites
In this exercise, you will brush-up the fundamental concepts of relational databases and SQL. If you haven't taken an introductory databases course (or want to refresh your knowledge) we recommend you to read the following:

Garcia-Molina, Ullman, Widom: Database Systems: The Complete Book. Pearson, 2. Edition, 2008. (Chapters 1, 2, 3, and 6) [Available in the ETH Library] [[Online]](https://ebookcentral.proquest.com/lib/ethz/detail.action?pq-origsite=primo&docID=5832965) [[Selected solutions]](http://infolab.stanford.edu/~ullman/dscbsols/sols.html).

Or have a look at the recordings from Information Systems for Engineers - ETH Zurich, available on [[YouTube]](https://www.youtube.com/c/GhislainFournysLectures).

## Normal Forms

* Only 1-3 NF, no Boyce–Codd normal form (BCNF) anymore, right?
* Confused me both when I first learned it in my undergrad and when I reviewed it for this course.
* Created some mnemonics and some very contrived examples which really helped me remember.
* That's what I'd like to briefly describe.

**Tips**: 
* Think of NFs as safety levels
* The higher the level is, the safer the DB application

### 1NF

**Mnemonic:**
1NF is mostly concerned with properties of single (1) cells.

**Counterexamples**

* Mixing data types

| name     | grade |
|----------|--------|
| Rachel   | 5      |
| Monica   | 6      |
| Joey     | Fail   |
| Chandler | 5      |

* No primariy key:

| name     | grade |
|----------|--------|
| Rachel   | 5      |
| Monica   | 6      |
| Joey     | 3      |
| Chandler | 5      |
| Monica   | 5      |

    Data inconsistency

* Repeating groups:

| name     | grades                          |
|----------|---------------------------------|
| Rachel   | math: 5, physics: 5 |
| Monica   | math: 6, physics: 6, biology: 6 |
| Joey     | math: 3, biology: 3 |
| Chandler | physics: 4, biology: 4 |



**NOTE:** How do we solve the 3rd?
* Why don't make the `grades` strings?
* One way is to flatten the table

#### Normalization

* Flattening to resolve repeating groups

| name   | course  | grade |
|--------|---------|-------|
| Monica | math    | 6     |
| Monica | physics | 6     |
| Monica | biology | 6     |
| Joey   | math    | 3     |
| Joey   | biology | 3     |

(**NOTE: 1NF is the most basic, violation of which would result in syntatic error in DB**)

#### 1NF rules
* Mixing data types in cells within the same column is not permitted
* Tables without a primary key are not permitted
* Repeating groups are not permitted
* Using row order to convey information is not permitted

### 2NF

* Adding school information

| name     | course  | grade | school |
|----------|---------|-------|--------|
| Monica   | math    | 6     | ETH    |
| Monica   | physics | 6     | ETH    |
| Monica   | biology | 6     | ETH    |
| Chandler | math    | 4     | EPFL   |

#### Deletion anomaly

* Delete row 4 because the grade was wrong:

| name     | course  | grade | school |
|----------|---------|-------|--------|
| Monica   | math    | 6     | ETH    |
| Monica   | physics | 6     | ETH    |
| Monica   | biology | 6     | ETH    |

*To which school does Chandler belong?*

#### Update anomaly

* Monica transferred to EPFL

| name     | course  | grade | school |
|----------|---------|-------|--------|
| Monica   | math    | 6     | EPFL   |
| Monica   | physics | 6     | EPFL   |
| Monica   | biology | 6     | **ETH**|

    Data inconsistency

**What caused these problems?**

#### 2NF rules
* Each non-key attribute must depend on the entire primary key

#### Normalization

* Primary key being a composite key: `{name, course}`
* Non-key attributes: `{grade, school}`

Checks:
* `{name, course} -> {grade}` ✅
* `{name} -> {school}` ❌

* Split tables (***key***)

| *name*   | *course*| grade |
|----------|---------|-------|
| Monica   | math    | 6     |
| Monica   | physics | 6     |
| Monica   | biology | 6     |
| Chandler | math    | 4     |

| *name*   |school  |
|----------|-------|
| Monica   | ETH |
| Chandler | EPFL   |


**Mnemonic:**
2NF mainly focuses on tables having composite key that consists of ≥**2** columns.

### 3NF

* Add rating

| *name*   | *course*| grade | rating |
|----------|---------|-------|--------|
| Monica   | math    | 6     | Good   |
| Monica   | physics | 6     | Good   |
| Monica   | biology | 6     | Good   |
| Chandler | math    | 4     | Pass   |  

* Chandler's math grade was wrong: 4 -> 6

| *name*   | *course*| grade | rating |
|----------|---------|-------|--------|
| Monica   | math    | 6     | Good   |
| Monica   | physics | 6     | Good   |
| Monica   | biology | 6     | Good   |
| Chandler | math    | 6     | **Pass**|

    Data inconsistency

#### Transitive dependency

* `{name, course} -> {grade}` ✅
* `{name, course} -> {grade} -> {rating}` ❌

#### 3NF rules
* Every non-key attribute in a table should depend on the key, the whole key, and nothing but the key.

#### Normalization

* Split the table (again)

| *name*   | *course*| grade |
|----------|---------|-------|
| Monica   | math    | 6     |
| Monica   | physics | 6     |
| Monica   | biology | 6     |
| Chandler | math    | 4     | 

| *grade* | rating |
|-------|--------|
| 6     | Good   |
| 4     | Pass   |

**Nemonic:** 3NF deals with transitive dependencies, which typically involve ≥**3** attributes.

#### BCNF
* Every **~non-key~** attribute in a table should depend on the key, the whole key, and nothing but the key.

(**NOTE: Haven't seen a 3NF table not being BCNF in real life**)

### Objectives of NFs

1. **Reduction of Redundancy:**
   Normalization helps eliminate redundant data within a database by organizing it into smaller, more manageable tables. Reducing redundancy ensures that each piece of information is stored in only one place, improving data consistency and saving storage space.

2. **Prevention of Update Anomalies:**
   Normalization helps prevent update anomalies that can occur when data is duplicated. Without normalization, updating one instance of duplicated data might result in inconsistencies or errors if the other instances are not updated correctly.

3. **Prevention of Insertion Anomalies:**
   Insertion anomalies can occur when attempting to insert new data into a table. Proper normalization ensures that all required information can be inserted without violating any constraints, thereby maintaining data integrity.

4. **Prevention of Deletion Anomalies:**
   Normalization helps prevent deletion anomalies, which can occur if deleting a piece of information unintentionally removes other related data that is valid and necessary.

5. **Simplified Modification Process:**
   With a normalized database, modifying the structure or content of the database is more straightforward. Changes can be made in one place without the need to update redundant data scattered across the database.

6. **Enhanced Readability and Understandability:**
   Normalized databases are typically organized in a logical and clear manner, making it easier for developers, analysts, and users to understand the database structure and relationships.

7. **Efficient Data Retrieval:**
   By breaking down data into smaller, logically related tables, normalization can improve data retrieval efficiency by reducing the need for redundant storage and enabling more efficient queries.

---

**In short, normalized tables are**
* easier to understand
* easier to enhance and extend
* projected from all kinds of anormalises

Overall, normal forms play a crucial role in ensuring that a database is well-structured, consistent, and efficient, which is essential for the reliable and effective management of data in various applications and systems.

### Origin of the data
You can find more information on the dataset in the following links
- [Discogs](https://www.discogs.com/)
- [Discogs XML data dumps](http://data.discogs.com/)

If you do not want to use Docker or it does not work you can download the dataset from this [link](https://cloud.inf.ethz.ch/s/DtjCHTLRHT39BRN/download/discogs.dump.xz), see `postgres-init.sh` to see how to import it)

In [1]:
server='db'
user='postgres'
password='example'
database='postgres'
connection_string=f'postgresql://{user}:{password}@{server}:5432/{database}'

In [2]:
%reload_ext sql
%sql $connection_string

In [3]:
%%sql
SELECT version();

 * postgresql://postgres:***@db:5432/postgres
1 rows affected.


version
"PostgreSQL 16.0 (Debian 16.0-1.pgdg120+1) on aarch64-unknown-linux-gnu, compiled by gcc (Debian 12.2.0-14) 12.2.0, 64-bit"


### Database Set-up
We will be once again working in the ExamMagicBox (you can find it in the following [[link]](http://please.add.link.here) if you have not downloaded it yet): please drag this Notebook in the folder. Just like last week, activate the docker container for the exercise sheet with `docker compose up`; please wait for the message `PostgreSQL init process complete; ready for start up` in the docker logs before proceeding! Alternatively you can start the Docker with `docker compose up -d` and wait for the command to execute: please note that you are creating the containers in the background this way. You can then type `docker compose down` when you are done.

As before, we set up our connection to the database and enable use of `%sql` and `%%sql`.

## Exercise 1: Explore the dataset
We want to first understand the dataset a bit better. You will find some queries below to help you explore the schema.

### List tables
The following query retrieves a list of tables in the database from a system table describing the current database.

In [4]:
%%sql 
SELECT table_name
FROM information_schema.tables
WHERE table_schema = 'public';

 * postgresql://postgres:***@db:5432/postgres
22 rows affected.


table_name
companies
jobs
employees02
badges
comments
inventory
posthistory
postlinks
posts
tags


### List attributes/columns
The following query retrieves a list of columns from the tables in the database we will be using.

In [5]:
%%sql 
SELECT table_name, column_name, data_type, is_nullable, ordinal_position
FROM information_schema.columns
WHERE table_schema = 'public' AND table_name IN ('artists', 'released_by', 'releases', 'tracks')
AND table_name NOT LIKE 'pg_%'
ORDER BY table_name, ordinal_position;

 * postgresql://postgres:***@db:5432/postgres
17 rows affected.


table_name,column_name,data_type,is_nullable,ordinal_position
artists,artist_id,integer,NO,1
artists,name,character varying,YES,2
artists,realname,text,YES,3
artists,profile,text,YES,4
artists,url,text,YES,5
released_by,release_id,integer,NO,1
released_by,artist_id,integer,NO,2
releases,release_id,integer,NO,1
releases,released,date,NO,2
releases,title,text,NO,3


### Have a look at the datasets
The following simple query gives the first 5 rows of the `artists` dataset

In [9]:
%%sql
SELECT * FROM artists LIMIT 5;

 * postgresql://postgres:***@db:5432/postgres
5 rows affected.


artist_id,name,realname,profile,url
1,The Persuader,Jesper Dahlbäck,None,None
2,Mr. James Barth & A.D.,Cari Lekebusch & Alexi Delano,None,None
3,Josh Wink,Joshua Winkelman,"After forming [l=Ovum Recordings] as an independent label in October 1994 with former partner [a=King Britt], Josh recorded the cult classic 'Liquid Summer'. He went on to release singles for a wide variety of revered European labels ranging from Belgium's [l=R & S Records] to England's [l=XL Recordings]. In 1995, Wink became one of the first DJ-producers to translate his hard work into mainstream success when he unleashed a string of classics including 'Don't Laugh'¸ 'I'm Ready' and 'Higher State of Consciousness' that topped charts worldwide. More recently he has had massive club hits such as 'How's Your Evening So Far' and 'Superfreak' but he has also gained a lot of attention trough his remixes for [a=FC Kahuna], [a=Paul Oakenfold], [a=Ladytron], [a=Clint Mansell], [a=Sting] and [a=Depeche Mode], among others.",http://www.joshwink.com/
4,Johannes Heil,Johannes Heil,"Electronic music producer, musician and live performer, born 3 February 1978 near the town of Bad Nauheim, Germany. Founder of [l=JH] and [l=Metatron Recordings].",http://johannes-heil.com/
5,Heiko Laux,Heiko Laux,German DJ and producer based in Berlin. He is the founder of [l=Kanzleramt].,http://www.heiko-laux.com


In [10]:
%%sql
SELECT * FROM tracks LIMIT 5;

 * postgresql://postgres:***@db:5432/postgres
5 rows affected.


release_id,position,title,duration,track_id
1,A,Östermalm,285,1
1,B1,Vasastaden,371,2
1,B2,Kungsholmen,169,3
1,C1,Södermalm,338,4
1,C2,Norrmalm,292,5


Naturally we could write similar queries to better understand each of the other tables.

### With what you now know about the datasets, try to answer the following questions

1. Which concepts are modelled in the dataset and how do they relate to each other? <b>Hint</b>: how do the tables connect logically?
2. Why do you think this shape (table) was chosen for the data and why not the other shapes? 
3. In which normal forms are the corresponding relations? 
4. How can we denormalise the data to make some queries more efficient? <b>Hint</b>: have a look at the queries in the next session of the exercises to see if adding some columns to some tables could reduce the need to `JOIN`.
5. What potential problems could result from adding redundancy?

**NOTE: Track is a single piece of music, while a release is a collection of tracks packaged together for distribution.**

**NOTE: Below is my answers, which could be different from the official solution.**

1. The dataset models the concepts of artists, releases (albums), and tracks. The artists table contains information about individual artists, identified by artist_id. The releases table contains information about albums, each identified by a release_id. The tracks table contains information about individual tracks, each identified by a track_id and associated with a specific release through the release_id. The released_by table connects the artists and releases tables by indicating which artist released which album.

2. This shape was chosen for the data to normalize it and reduce redundancy. By separating the data into distinct tables (artists, releases, and tracks), each piece of information is stored only once, reducing the chance of inconsistencies. For example, instead of repeating the artist's information for each track they've created, the data model simply references the artist's ID.

3. The relations are in at least the third normal form (3NF). In 1NF, each table has a primary key (artist_id, release_id, track_id), and all columns are atomic. In 2NF, there are no partial dependencies (no non-key attribute depends on a part of the primary key). In 3NF, there are no transitive dependencies (non-key attributes do not depend on other non-key attributes).

4. To make some queries more efficient, we could denormalize the data by adding some redundancy. For instance, if many queries join the artists and releases tables, we could add an artist_name column to the releases table. Similarly, if many queries join the releases and tracks tables, we could add release_title and genre columns to the tracks table.

5. Adding redundancy could lead to several potential problems. First, it increases the space required to store the data. Second, it can lead to inconsistencies if not managed properly; if an artist's name is changed in the artists table but not in the releases table, for example, the data will be inconsistent. Lastly, it can make updates more complicated, as changes need to be made in multiple places.

## Exercise 2: SQL warm-up
Let us begin with several SQL queries to ease us back into the language.

#### 1. Retrieve the artists with the name of 'Coldplay'.

In [11]:
%%sql

SELECT *
FROM artists 
WHERE name = 'Coldplay' OR realname = 'Coldplay'

 * postgresql://postgres:***@db:5432/postgres
1 rows affected.


artist_id,name,realname,profile,url
29735,Coldplay,None,"Coldplay is an English rock band from London, England. They've been a band since January 16, 1998 when they lost a demotape competition on XFM in London. Philip Christopher Harvey is the band's manager. [b][u]Line-up:[/u][/b] Jonny Buckland (Jonathan Mark Buckland) - Guitar Will Champion (William Champion) - Drums Guy Berryman (Guy Rupert Berryman) - Bass Chris Martin (Christopher Anthony John Martin) - Vocals",http://coldplay.com/


#### 2. List the titles of all releases by that artist in alphabetical order.
<b>Hint</b>: Ignore the fact that different relases can have the same title.

**NOTE**
> The USING clause is a shorthand that allows you to take advantage of the specific situation where both sides of the join use the same name for the joining column(s). It takes a comma-separated list of the shared column names and forms a join condition that includes an equality comparison for each one. For example, joining T1 and T2 with USING (a, b) produces the join condition ON T1.a = T2.a AND T1.b = T2.b.

**NOTE: first filter by `Coldplay` before joining**

In [31]:
%%time
%%sql

SELECT DISTINCT R.title
FROM artists as A
JOIN released_by as RB on A.name = 'Coldplay' AND A.artist_id = RB.artist_id
JOIN releases as R USING(release_id)
ORDER BY R.title

 * postgresql://postgres:***@db:5432/postgres
40 rows affected.
CPU times: user 2.39 ms, sys: 2.01 ms, total: 4.41 ms
Wall time: 258 ms


title
Acoustic
A Rush Of Blood To The Head
Boot Of Sound
Brothers & Sisters
Clocks
Clocks...
Clocks / Chime Trance Remixes
Clocks (Cosmos Rmx)
Clocks (Dean Coleman Remix)
Clocks (Planet Rockers Remixes)


#### 3. How many tracks from 'Coldplay' have position '1'?

In [44]:
%%sql

SELECT COUNT(*)
FROM artists as A
JOIN released_by as RB on A.name = 'Coldplay' AND A.artist_id = RB.artist_id
--JOIN releases as R USING(release_id)
JOIN tracks as T USING(release_id) 
WHERE T.position = '1'

 * postgresql://postgres:***@db:5432/postgres
1 rows affected.


count
32


#### 4. What is the average track duration?

In [41]:
%%sql

SELECT AVG(T.duration)
FROM tracks as T

 * postgresql://postgres:***@db:5432/postgres
1 rows affected.


avg
324.5957277321236343


#### 5. How many artists have released tracks longer than twice the average?

In [54]:
%%sql

SELECT COUNT(DISTINCT A.artist_id)
FROM artists as A
JOIN released_by USING(artist_id)
JOIN tracks as T USING(release_id)
WHERE T.duration > 2 * (
    SELECT AVG(TT.duration)
    FROM tracks as TT
)

 * postgresql://postgres:***@db:5432/postgres
1 rows affected.


count
18894


## Exercise 3: more SQL
We will now see more complex SQL queries.

#### 1. What is the title of the album (release) from 'Coldplay' with the most number of tracks?

**NOTE:**

**Divide and conquer**
* Reuse 2.3
* Gradually build up the queries inside out

In [76]:
%%sql

SELECT * 
FROM (
    SELECT R.title, COUNT(T.track_id)
    FROM artists as A
    JOIN released_by as RB on A.name = 'Coldplay' AND A.artist_id = RB.artist_id
    JOIN releases as R USING(release_id)
    JOIN tracks as T USING(release_id)
    GROUP BY R.release_id, R.title
) as C
ORDER BY C.count DESC
LIMIT 1

 * postgresql://postgres:***@db:5432/postgres
1 rows affected.


title,count
X&Y (Special Dutch Edition),19


#### 2. How many artists have released an album with total track duration above twice the average total track duration?

<b>Hint</b>: this is not the same as exercise 2.5 since we are lookong at the <b>total</b> track duration of the album.

In [108]:
%%sql

WITH total_release_duration as (
    SELECT T.release_id, SUM(T.duration)
    FROM tracks as T
    GROUP BY T.release_id 
)

SELECT COUNT(DISTINCT A.artist_id)
FROM artists as A
JOIN released_by USING(artist_id)
JOIN total_release_duration as S USING(release_id)
WHERE S.sum > 2 * (
    SELECT AVG(total_release_duration.sum)
    FROM total_release_duration
)


 * postgresql://postgres:***@db:5432/postgres
1 rows affected.


count
16426


#### 3. How many artists have both a release with a track longer than twice the average and one with total duration longer than twice the average?

<b>Hint</b>: you can use `INTERSECT` or `EXISTS` to write your query.

In [114]:
%%sql

-- 2nd part (reuse 3.2 w/o counting)

WITH total_release_duration as (
    SELECT T.release_id, SUM(T.duration)
    FROM tracks as T
    GROUP BY T.release_id 
)

SELECT DISTINCT A.artist_id
FROM artists as A
JOIN released_by USING(artist_id)
JOIN total_release_duration as S USING(release_id)
WHERE S.sum > 2 * (
    SELECT AVG(total_release_duration.sum)
    FROM total_release_duration
)

-- Answer the 1st part

INTERSECT

SELECT DISTINCT A.artist_id
FROM artists as A
JOIN released_by USING(artist_id)
JOIN tracks as T USING(release_id)
WHERE T.duration > 2 * (
    SELECT AVG(TT.duration)
    FROM tracks as TT
)


 * postgresql://postgres:***@db:5432/postgres
8146 rows affected.


artist_id
1552
844372
1373
152359
161
25110
215804
32628
420323
327330


**NOTE: Count the results**

In [115]:
%%sql

-- 2nd part (reuse 3.2 w/o counting)

WITH total_release_duration as (
    SELECT T.release_id, SUM(T.duration)
    FROM tracks as T
    GROUP BY T.release_id 
)

SELECT COUNT(*)
FROM (
    SELECT DISTINCT A.artist_id
    FROM artists as A
    JOIN released_by USING(artist_id)
    JOIN total_release_duration as S USING(release_id)
    WHERE S.sum > 2 * (
        SELECT AVG(total_release_duration.sum)
        FROM total_release_duration
    )

    -- Answer the 1st part

    INTERSECT

    SELECT DISTINCT A.artist_id
    FROM artists as A
    JOIN released_by USING(artist_id)
    JOIN tracks as T USING(release_id)
    WHERE T.duration > 2 * (
        SELECT AVG(TT.duration)
        FROM tracks as TT
    )
)

 * postgresql://postgres:***@db:5432/postgres
1 rows affected.


count
8146


#### 4. Show the artists have more than 200 releases in total but have no releases with the genre 'Pop' in reversed alphabetical order.

> NATURAL JOIN uses all the columns with matching names and datatypes to join the tables. The USING Clause can be used to specify only those columns that should be used for an EQUIJOIN

In [155]:
%%sql

-- Step 1: artist_id -> release_count + filter by 200

WITH productive_artist as (
    SELECT A.name, A.artist_id
    FROM artists as A
    JOIN released_by as RB USING(artist_id)
    GROUP BY artist_id
    HAVING COUNT(release_id) > 200
)


-- Step 2: Get productive artists without releases of 'Pop'

SELECT PA.name
FROM productive_artist as PA
WHERE NOT EXISTS (
    SELECT *
    FROM artists as A
    JOIN released_by USING(artist_id)
    JOIN releases as R USING(release_id)
    WHERE R.genre = 'Pop' and PA.artist_id = A.artist_id
)
ORDER BY PA.name DESC


 * postgresql://postgres:***@db:5432/postgres
35 rows affected.


name
Yello
WestBam
Underworld
U2
The Shamen
The Prodigy
The Human League
The Cure
The Chemical Brothers
The Beatles


**Using `IN`**

In [156]:
%%sql

-- Step 1: artist_id -> release_count + filter by 200

WITH productive_artist as (
    SELECT A.name, A.artist_id
    FROM artists as A
    JOIN released_by as RB USING(artist_id)
    GROUP BY artist_id
    HAVING COUNT(release_id) > 200
)


-- Step 2: Get productive artists without releases of 'Pop'

SELECT PA.name
FROM productive_artist as PA
WHERE PA.artist_id not in (
    SELECT A.artist_id
    FROM artists as A
    JOIN released_by USING(artist_id)
    JOIN releases as R USING(release_id)
    WHERE R.genre = 'Pop'
)
ORDER BY PA.name DESC


 * postgresql://postgres:***@db:5432/postgres
35 rows affected.


name
Yello
WestBam
Underworld
U2
The Shamen
The Prodigy
The Human League
The Cure
The Chemical Brothers
The Beatles


## Exercise 4: Discuss query patterns and language features of SQL
1. What patterns did you use in many of the queries above? 

2. What is the usual pattern of an SQL query? Which operations happen pre-grouping and which ones post-grouping?

3. What makes SQL a declarative language and what advantages does that have?

4. What makes SQL a functional language and what advantages does that have?

5. How would the denormalization we talked about previously simplify the queries?

1. **Patterns Used in SQL Queries:**
   In SQL queries, common patterns include:
   - **SELECT-FROM-WHERE**: Selecting specific columns from a table based on specified conditions.
   - **JOIN**: Combining data from multiple tables based on related columns.
   - **GROUP BY**: Aggregating data and performing operations on groups of data.
   - **ORDER BY**: Sorting the results based on specified columns.
   - **SUBQUERIES**: Nesting one query within another to retrieve more complex results.
   - **UNION, INTERSECT, EXCEPT**: Combining, finding commonalities, or finding differences between result sets.

2. **Usual Pattern of an SQL Query:**
   The usual pattern of an SQL query is:
   ```sql
   SELECT [columns]
   FROM [table(s)]
   WHERE [conditions]
   GROUP BY [columns]
   HAVING [conditions after grouping]
   ORDER BY [columns]
   ```
   Operations like filtering (`WHERE`), grouping (`GROUP BY`), and sorting (`ORDER BY`) typically happen before the data is presented to the user, while operations involving aggregation (`SUM`, `COUNT`, etc.) usually occur post-grouping.

3. **SQL as a Declarative Language:**
   SQL is declarative, meaning that you specify what you want to retrieve or manipulate without specifying how to achieve it. Users describe the desired results, and the SQL engine determines the most efficient way to execute the query. This provides advantages such as simplicity, abstraction, and focus on what is needed rather than the implementation details.

4. **SQL as a Functional Language:**
   SQL can exhibit functional programming characteristics, particularly with operations like `SUM`, `COUNT`, `MIN`, `MAX`, etc., which treat database tables and result sets as collections of data. It allows for data transformation without modifying the original data, adhering to functional programming principles. This supports immutability and can lead to more maintainable and understandable code.

5. **Denormalization and Query Simplification:**
   Denormalization involves adding redundant data to a table to reduce the need for joins and simplify queries. In the context of the database discussed earlier, denormalization could involve adding redundant artist information (e.g., artist name) to the releases table. This would simplify queries related to releases, eliminating the need to join with the artists table to retrieve artist details, thereby improving query efficiency and readability.

## Exercise 5: Limits of SQL (optional)
Explain what the following query does.
<b>Hints</b>: The query treats the data as if it was in graph shape.

In [ ]:
%%sql
WITH RECURSIVE
    X AS (SELECT 3 AS Value),
    artist_releases AS (
        SELECT artists.artist_id, artists.name, releases.release_id, releases.title
        FROM artists, released_by, releases
        WHERE artists.artist_id = released_by.artist_id
        AND released_by.release_id = releases.release_id
    ),
    collaborations AS (
        SELECT DISTINCT ar1.artist_id AS left_id, ar1.name AS left_name, 
                ar2.artist_id AS right_id, ar2.name AS right_name, 1 AS distance
        FROM artist_releases AS ar1, artist_releases AS ar2
        WHERE ar1.release_id = ar2.release_id
        AND ar1.artist_id != ar2.artist_id
    ),
    X_hop_collaborations AS (
        SELECT * FROM collaborations  -- base case
        UNION
        SELECT c1.left_id, c1.left_name, c2.right_id, c2.right_name, c1.distance + 1 AS distance
        FROM X_hop_collaborations AS c1
        JOIN collaborations c2 ON c1.right_id = c2.left_id
        WHERE c1.distance < (SELECT * FROM X)
    )
SELECT * 
FROM X_hop_collaborations
WHERE left_name = 'Coldplay'
ORDER BY distance, right_name;